**# Data Gathering**
only use data from last season to predict next season, as earlier seasons may give us bad estimates as quality of teams may have changed.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm

loc = 'Dataset//'

data1 = pd.read_csv(loc + 'E0 (1).csv')

df = [data1]


for ind, data in enumerate(df):
    data['Season'] = "{}/{}".format(18 - ind, 19 - ind)

col_list = ['HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'HS',
            'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'Season']

all_data = pd.concat(df, sort='False')
all_data = all_data[col_list]

def result(HG, AG):
    if HG > AG:
        return -1
    elif HG == AG:
        return 0
    elif HG < AG:
        return 1

all_data['Result'] = all_data.apply(lambda row: result(row.FTHG, row.FTAG), axis = 1) 
print(all_data.shape)
all_data

(380, 18)


/var/folders/qk/cl2c3dyd16908blk55j0pfhr0000gn/T/ipykernel_77099/2239054788.py:27: FutureWarning: Passing non boolean values for sort is deprecated and will error in a future version!
  all_data = pd.concat(df, sort='False')


,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST,HF,AF,HC,AC,Season,Result
0,Brentford,Arsenal,2,0,H,1,0,H,8,22,3,4,12,8,2,5,18/19,-1
1,Man United,Leeds,5,1,H,1,0,H,16,10,8,3,11,9,5,4,18/19,-1
2,Burnley,Brighton,1,2,A,1,0,H,14,14,3,8,10,7,7,6,18/19,1
3,Chelsea,Crystal Palace,3,0,H,2,0,H,13,4,6,1,15,11,5,2,18/19,-1
4,Everton,Southampton,3,1,H,0,1,A,14,6,6,3,13,15,6,8,18/19,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,Crystal Palace,Man United,1,0,H,1,0,H,6,10,3,4,12,22,3,6,18/19,-1
376,Leicester,Southampton,4,1,H,0,0,D,12,7,6,2,10,5,3,3,18/19,-1
377,Liverpool,Wolves,3,1,H,1,1,D,29,7,8,5,6,3,5,3,18/19,-1
378,Man City,Aston Villa,3,2,H,0,1,A,24,4,5,2,5,11,13,1,18/19,-1


**# Data Wrangling**

In [2]:
home_data = all_data.groupby('HomeTeam').sum()
away_data = all_data.groupby('AwayTeam').sum()
home_data['HAS'] = home_data['FTHG']/19 #home avg scores
home_data['HAC'] = home_data['FTAG']/19 # Home Average Conceded
away_data['AAS'] = away_data['FTAG']/19 # Away average Scored
away_data['AAC'] = away_data['FTHG']/19 # Away Average Conceded
all_data_backup = all_data
team_data = pd.DataFrame()
team_data['Team'] = home_data.index

team_data = team_data.merge(home_data['HAS'], left_on='Team', right_index=True)
team_data = team_data.merge(home_data['HAC'], left_on='Team', right_index=True)

team_data = team_data.merge(away_data['AAS'], left_on='Team', right_index=True)
team_data = team_data.merge(away_data['AAC'], left_on='Team', right_index=True)

team_data.set_index("Team", inplace=True)
last_season_table= pd.DataFrame(index=team_data.index)
scaler = MinMaxScaler()
team_data = pd.DataFrame(scaler.fit_transform(team_data), columns=team_data.columns, index=team_data.index)
team_data

/var/folders/qk/cl2c3dyd16908blk55j0pfhr0000gn/T/ipykernel_77099/4100648613.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  home_data = all_data.groupby('HomeTeam').sum()
/var/folders/qk/cl2c3dyd16908blk55j0pfhr0000gn/T/ipykernel_77099/4100648613.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  away_data = all_data.groupby('AwayTeam').sum()


,HAS,HAC,AAS,AAC
Team,,,,
Arsenal,0.500000,0.216216,0.441176,0.62500
Aston Villa,0.369565,0.540541,0.352941,0.43750
Brentford,0.217391,0.324324,0.441176,0.75000
Brighton,0.152174,0.378378,0.352941,0.31250
Burnley,0.130435,0.432432,0.147059,0.53125
Chelsea,0.543478,0.351351,0.823529,0.00000
Crystal Palace,0.326087,0.216216,0.352941,0.56250
Everton,0.326087,0.432432,0.147059,0.93750
Leeds,0.152174,0.783784,0.352941,0.93750


In [3]:
col_list2 = ['HomeTeam', 'AwayTeam']
data_prep = all_data[col_list2]

data_prep = data_prep.merge(team_data['HAS'], left_on='HomeTeam', right_index=True, how='left')
data_prep = data_prep.merge(team_data['HAC'], left_on='HomeTeam', right_index=True, how='left') 
data_prep = data_prep.merge(team_data['AAS'], left_on='AwayTeam', right_index=True, how='left') 
data_prep = data_prep.merge(team_data['AAC'], left_on='AwayTeam', right_index=True, how='left') 

data_prep['HAtS'] = (data_prep['HAS'] * data_prep['AAC']); # Home Attack Strength
data_prep['AAtS'] = (data_prep['HAC'] * data_prep['AAS']); # Away Attack Strength

print(data_prep.shape)

data_prep = data_prep.merge(all_data['Result'], left_index=True, right_index=True)

fixture_list = data_prep
data_prep

(380, 8)


,HomeTeam,AwayTeam,HAS,HAC,AAS,AAC,HAtS,AAtS,Result
0,Brentford,Arsenal,0.217391,0.324324,0.441176,0.62500,0.135870,0.143084,-1
1,Man United,Leeds,0.434783,0.351351,0.352941,0.93750,0.407609,0.124006,-1
2,Burnley,Brighton,0.130435,0.432432,0.352941,0.31250,0.040761,0.152623,1
3,Chelsea,Crystal Palace,0.543478,0.351351,0.352941,0.56250,0.305707,0.124006,-1
4,Everton,Southampton,0.326087,0.432432,0.264706,1.00000,0.326087,0.114467,-1
...,...,...,...,...,...,...,...,...,...
375,Crystal Palace,Man United,0.326087,0.216216,0.411765,0.75000,0.244565,0.089030,-1
376,Leicester,Southampton,0.478261,0.378378,0.264706,1.00000,0.478261,0.100159,-1
377,Liverpool,Wolves,0.804348,0.000000,0.205882,0.21875,0.175951,0.000000,-1
378,Man City,Aston Villa,1.000000,0.162162,0.352941,0.43750,0.437500,0.057234,-1


In [4]:
rm_col_list = ['HomeTeam','AwayTeam']
col_list = data_prep.columns.values.tolist()
col_list = [x for x in col_list if x not in rm_col_list]

to_use = data_prep[col_list]
X = [x for x in col_list if x != 'Result']
Y = [x for x in col_list if x == 'Result']

to_use

,HAS,HAC,AAS,AAC,HAtS,AAtS,Result
0,0.217391,0.324324,0.441176,0.62500,0.135870,0.143084,-1
1,0.434783,0.351351,0.352941,0.93750,0.407609,0.124006,-1
2,0.130435,0.432432,0.352941,0.31250,0.040761,0.152623,1
3,0.543478,0.351351,0.352941,0.56250,0.305707,0.124006,-1
4,0.326087,0.432432,0.264706,1.00000,0.326087,0.114467,-1
...,...,...,...,...,...,...,...
375,0.326087,0.216216,0.411765,0.75000,0.244565,0.089030,-1
376,0.478261,0.378378,0.264706,1.00000,0.478261,0.100159,-1
377,0.804348,0.000000,0.205882,0.21875,0.175951,0.000000,-1
378,1.000000,0.162162,0.352941,0.43750,0.437500,0.057234,-1


In [5]:
X_data = to_use.drop('Result', axis=1)
Y_data = to_use['Result']

model = LogisticRegression(max_iter=10000)
model.fit(X_data, Y_data)
model.score(X_data, Y_data)

0.5815789473684211

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size=0.2, random_state=42)

model1 = LogisticRegression(max_iter=10000)
model1.fit(X_train, Y_train)

LogisticRegression(max_iter=10000)

In [7]:
predicted = model1.predict(X_test)
predicted

array([-1,  1, -1, -1,  1, -1,  1,  1, -1,  1, -1, -1,  1, -1,  1, -1,  1,
        1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1,  1,  1, -1,  1,  1, -1,
       -1, -1,  1, -1,  1, -1, -1, -1, -1, -1,  1, -1, -1,  1,  1,  1,  1,
        1,  1, -1,  1, -1,  1,  1,  1, -1, -1,  1,  1, -1,  1, -1, -1, -1,
        1, -1, -1, -1,  1,  1, -1,  1])

In [8]:
acc_score = accuracy_score(Y_test, predicted)
print("Accuracy score: " + str(acc_score))
scores = cross_val_score(model1, X_train, Y_train, cv=10)
print("Cross val scores")
print(scores)

Accuracy score: 0.631578947368421
Cross val scores
[0.48387097 0.58064516 0.70967742 0.51612903 0.5        0.6
 0.63333333 0.53333333 0.5        0.53333333]


In [9]:
score_train = model1.score(X_train, Y_train)
score_test = model1.score(X_test, Y_test)

print("Training set accuracy: ", '%.3f'%(score_train))
print("Test set accuracy: ", '%.3f'%(score_test))

Training set accuracy:  0.572
Test set accuracy:  0.632


In [10]:
predicted = model1.predict(X_data)
predicted
predictedDF = pd.DataFrame(predicted, columns=['Predicted'])
predictedDF.head()

,Predicted
0,-1
1,-1
2,1
3,-1
4,-1


In [11]:
to_use = to_use.merge(predictedDF, left_index=True, right_index=True)
fixture_list = fixture_list.merge(predictedDF, left_index=True, right_index=True)
fixture_list

,HomeTeam,AwayTeam,HAS,HAC,AAS,AAC,HAtS,AAtS,Result,Predicted
0,Brentford,Arsenal,0.217391,0.324324,0.441176,0.62500,0.135870,0.143084,-1,-1
1,Man United,Leeds,0.434783,0.351351,0.352941,0.93750,0.407609,0.124006,-1,-1
2,Burnley,Brighton,0.130435,0.432432,0.352941,0.31250,0.040761,0.152623,1,1
3,Chelsea,Crystal Palace,0.543478,0.351351,0.352941,0.56250,0.305707,0.124006,-1,-1
4,Everton,Southampton,0.326087,0.432432,0.264706,1.00000,0.326087,0.114467,-1,-1
...,...,...,...,...,...,...,...,...,...,...
375,Crystal Palace,Man United,0.326087,0.216216,0.411765,0.75000,0.244565,0.089030,-1,-1
376,Leicester,Southampton,0.478261,0.378378,0.264706,1.00000,0.478261,0.100159,-1,-1
377,Liverpool,Wolves,0.804348,0.000000,0.205882,0.21875,0.175951,0.000000,-1,-1
378,Man City,Aston Villa,1.000000,0.162162,0.352941,0.43750,0.437500,0.057234,-1,-1


In [12]:
last_season_table['Actual Points'] = 0 
last_season_table['Predicted Points'] = 0 

def make_actual_table(x, ht, at, which):
    if x == -1:
        last_season_table.loc[ht, which] += 3
    elif x == 0:
        last_season_table.loc[ht, which] += 1
        last_season_table.loc[at, which] += 1
    elif x == 1:
        last_season_table.loc[at, which] += 3

fixture_list.apply(lambda x: make_actual_table(x.Result, x.HomeTeam, x.AwayTeam, 'Actual Points'), axis=1)
fixture_list.apply(lambda x: make_actual_table(x.Predicted, x.HomeTeam, x.AwayTeam, 'Predicted Points'), axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
375    None
376    None
377    None
378    None
379    None
Length: 380, dtype: object

In [13]:
# 21/22 Season actual table
actual_index = last_season_table.sort_values(by='Actual Points', ascending=0).index
predicted_index = last_season_table.sort_values(by='Predicted Points', ascending=0).index

last_season_table.sort_values(by='Actual Points', ascending=0)

,Actual Points,Predicted Points
Team,,
Man City,93,111
Liverpool,92,111
Chelsea,74,102
Tottenham,71,93
Arsenal,69,66
Man United,58,57
West Ham,56,78
Leicester,52,63
Brighton,51,57


Actual vs predicted for 21/22 season

In [14]:
def acc(act, prd):
    count = 0
    for i, val in enumerate(act):
        if str(act[i]) == str(prd[i]):
            count += 1
    print("Accuracy of Predicted Table to Accurate Table " + str((count/20)*100) + "%")

acc(actual_index,predicted_index)

# 18/19 Season predicted table
last_season_table.sort_values(by='Predicted Points', ascending =0)

#got a few top teams and bad teams right

Accuracy of Predicted Table to Accurate Table 30.0%


,Actual Points,Predicted Points
Team,,
Liverpool,92,111
Man City,93,111
Chelsea,74,102
Tottenham,71,93
West Ham,56,78
Arsenal,69,66
Crystal Palace,48,63
Leicester,52,63
Aston Villa,45,63


In [15]:
curr_season = pd.read_csv(loc + "E0.csv")
curr_season['Result'] = curr_season.apply(lambda row: result(row.FTHG, row.FTAG), axis = 1) 
curr_season.append(all_data, sort='False')
curr_season

/var/folders/qk/cl2c3dyd16908blk55j0pfhr0000gn/T/ipykernel_77099/1749113250.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  curr_season.append(all_data, sort='False')
/var/folders/qk/cl2c3dyd16908blk55j0pfhr0000gn/T/ipykernel_77099/1749113250.py:3: FutureWarning: Passing non boolean values for sort is deprecated and will error in a future version!
  curr_season.append(all_data, sort='False')


,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,Result
0,E0,05/08/2022,20:00,Crystal Palace,Arsenal,0,2,A,0,1,...,0.50,2.09,1.84,2.04,1.88,2.09,1.88,2.03,1.85,1
1,E0,06/08/2022,12:30,Fulham,Liverpool,2,2,D,1,0,...,1.75,1.90,2.03,1.91,2.02,2.01,2.06,1.89,1.99,0
2,E0,06/08/2022,15:00,Bournemouth,Aston Villa,2,0,H,1,0,...,0.50,1.93,2.00,1.93,2.00,1.94,2.04,1.88,2.00,-1
3,E0,06/08/2022,15:00,Leeds,Wolves,2,1,H,1,1,...,-0.25,2.08,1.85,2.10,1.84,2.14,1.87,2.08,1.81,-1
4,E0,06/08/2022,15:00,Newcastle,Nott'm Forest,2,0,H,0,0,...,-1.00,1.97,1.96,1.99,1.93,2.19,1.97,2.03,1.86,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,E0,28/05/2023,16:30,Everton,Bournemouth,1,0,H,0,0,...,-1.00,2.02,1.77,2.10,1.81,2.17,1.92,2.03,1.83,-1
376,E0,28/05/2023,16:30,Leeds,Tottenham,1,4,A,0,1,...,0.25,1.84,2.06,1.83,2.10,1.90,2.14,1.81,2.06,1
377,E0,28/05/2023,16:30,Leicester,West Ham,2,1,H,1,0,...,-0.25,1.75,2.05,1.85,2.06,1.90,2.16,1.82,2.04,-1
378,E0,28/05/2023,16:30,Man United,Fulham,2,1,H,1,1,...,-1.25,1.98,1.92,1.98,1.93,2.07,1.98,1.97,1.89,-1


Data wrangling for new season

In [16]:
curr_home_data = curr_season.groupby('HomeTeam').sum()
curr_away_data = curr_season.groupby('AwayTeam').sum()
curr_home_data['HAS'] = curr_home_data['FTHG'] / 19 # Home Average Scores
curr_home_data['HAC'] = curr_home_data['FTAG'] / 19 # Home Average Conceded
curr_away_data['AAS'] = curr_away_data['FTAG'] / 19 # Away average Scored
curr_away_data['AAC'] = curr_away_data['FTHG'] / 19 # Away Average Conceded
curr_season_backup = curr_season
curr_team_data = pd.DataFrame()
curr_team_data['Team'] = curr_home_data.index

curr_team_data = curr_team_data.merge(curr_home_data['HAS'], left_on='Team', right_index=True)
curr_team_data = curr_team_data.merge(curr_home_data['HAC'], left_on='Team', right_index=True)

curr_team_data = curr_team_data.merge(curr_away_data['AAS'], left_on='Team', right_index=True)
curr_team_data = curr_team_data.merge(curr_away_data['AAC'], left_on='Team', right_index=True)

curr_team_data.set_index("Team", inplace=True)
curr_season_table= pd.DataFrame(index=curr_team_data.index)
# curr_team_data = team_data.merge(curr_team_data, left_index=True, right_index=True, how='outer')
# curr_team_data = team_data.merge(curr_team_data, on=['HAS', 'HAC', 'AAS', 'AAC'],  how='outer')

curr_team_data = pd.DataFrame(scaler.fit_transform(curr_team_data), columns=curr_team_data.columns, index=curr_team_data.index)
curr_team_data

/var/folders/qk/cl2c3dyd16908blk55j0pfhr0000gn/T/ipykernel_77099/1405902019.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  curr_home_data = curr_season.groupby('HomeTeam').sum()
/var/folders/qk/cl2c3dyd16908blk55j0pfhr0000gn/T/ipykernel_77099/1405902019.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  curr_away_data = curr_season.groupby('AwayTeam').sum()


,HAS,HAC,AAS,AAC
Team,,,,
Arsenal,0.840909,0.555556,1.000000,0.071429
Aston Villa,0.386364,0.407407,0.291667,0.321429
Bournemouth,0.090909,0.666667,0.250000,0.964286
Brentford,0.431818,0.296296,0.500000,0.428571
Brighton,0.477273,0.407407,1.000000,0.571429
Chelsea,0.090909,0.333333,0.291667,0.428571
Crystal Palace,0.113636,0.481481,0.333333,0.357143
Everton,0.000000,0.629630,0.291667,0.500000
Fulham,0.340909,0.703704,0.541667,0.285714


In [17]:
col_list2 = ['HomeTeam', 'AwayTeam']
data_prep = curr_season[col_list2]

data_prep = data_prep.merge(curr_team_data['HAS'], left_on='HomeTeam', right_index=True, how='left')
data_prep = data_prep.merge(curr_team_data['HAC'], left_on='HomeTeam', right_index=True, how='left') 
data_prep = data_prep.merge(curr_team_data['AAS'], left_on='AwayTeam', right_index=True, how='left') 
data_prep = data_prep.merge(curr_team_data['AAC'], left_on='AwayTeam', right_index=True, how='left') 

data_prep['HAtS'] = data_prep['HAS'] * data_prep['AAC']; # Home Attack Strength
data_prep['AAtS'] = data_prep['HAC'] * data_prep['AAS']; # Away Attack Strength

print(data_prep.shape)

data_prep = data_prep.merge(curr_season['Result'], left_index=True, right_index=True)

fixture_list = data_prep
data_prep

(380, 8)


,HomeTeam,AwayTeam,HAS,HAC,AAS,AAC,HAtS,AAtS,Result
0,Crystal Palace,Arsenal,0.113636,0.481481,1.000000,0.071429,0.008117,0.481481,1
1,Fulham,Liverpool,0.340909,0.703704,0.750000,0.500000,0.170455,0.527778,0
2,Bournemouth,Aston Villa,0.090909,0.666667,0.291667,0.321429,0.029221,0.194444,-1
3,Leeds,Wolves,0.227273,1.000000,0.041667,0.785714,0.178571,0.041667,-1
4,Newcastle,Nott'm Forest,0.454545,0.148148,0.000000,1.000000,0.454545,0.000000,-1
...,...,...,...,...,...,...,...,...,...
375,Everton,Bournemouth,0.000000,0.629630,0.250000,0.964286,0.000000,0.157407,-1
376,Leeds,Tottenham,0.227273,1.000000,0.916667,0.785714,0.178571,0.916667,1
377,Leicester,West Ham,0.159091,0.629630,0.208333,0.535714,0.085227,0.131173,-1
378,Man United,Fulham,0.454545,0.000000,0.541667,0.285714,0.129870,0.000000,-1


In [18]:
rm_col_list = ['HomeTeam','AwayTeam']
col_list = data_prep.columns.values.tolist()
col_list = [x for x in col_list if x not in rm_col_list]

to_use = data_prep[col_list]
X = [x for x in col_list if x != 'Result']
Y = [x for x in col_list if x == 'Result']

X_data = to_use.drop('Result', axis=1)
Y_data = to_use['Result']

to_use

,HAS,HAC,AAS,AAC,HAtS,AAtS,Result
0,0.113636,0.481481,1.000000,0.071429,0.008117,0.481481,1
1,0.340909,0.703704,0.750000,0.500000,0.170455,0.527778,0
2,0.090909,0.666667,0.291667,0.321429,0.029221,0.194444,-1
3,0.227273,1.000000,0.041667,0.785714,0.178571,0.041667,-1
4,0.454545,0.148148,0.000000,1.000000,0.454545,0.000000,-1
...,...,...,...,...,...,...,...
375,0.000000,0.629630,0.250000,0.964286,0.000000,0.157407,-1
376,0.227273,1.000000,0.916667,0.785714,0.178571,0.916667,1
377,0.159091,0.629630,0.208333,0.535714,0.085227,0.131173,-1
378,0.454545,0.000000,0.541667,0.285714,0.129870,0.000000,-1


In [19]:
predicted = model1.predict(X_data)
predictedDF = pd.DataFrame(predicted, columns=['Predicted'])
predictedDF.head()

,Predicted
0,1
1,1
2,1
3,-1
4,-1


In [20]:
to_use = to_use.merge(predictedDF, left_index=True, right_index=True)
fixture_list = fixture_list.merge(predictedDF, left_index=True, right_index=True)
fixture_list

,HomeTeam,AwayTeam,HAS,HAC,AAS,AAC,HAtS,AAtS,Result,Predicted
0,Crystal Palace,Arsenal,0.113636,0.481481,1.000000,0.071429,0.008117,0.481481,1,1
1,Fulham,Liverpool,0.340909,0.703704,0.750000,0.500000,0.170455,0.527778,0,1
2,Bournemouth,Aston Villa,0.090909,0.666667,0.291667,0.321429,0.029221,0.194444,-1,1
3,Leeds,Wolves,0.227273,1.000000,0.041667,0.785714,0.178571,0.041667,-1,-1
4,Newcastle,Nott'm Forest,0.454545,0.148148,0.000000,1.000000,0.454545,0.000000,-1,-1
...,...,...,...,...,...,...,...,...,...,...
375,Everton,Bournemouth,0.000000,0.629630,0.250000,0.964286,0.000000,0.157407,-1,1
376,Leeds,Tottenham,0.227273,1.000000,0.916667,0.785714,0.178571,0.916667,1,1
377,Leicester,West Ham,0.159091,0.629630,0.208333,0.535714,0.085227,0.131173,-1,1
378,Man United,Fulham,0.454545,0.000000,0.541667,0.285714,0.129870,0.000000,-1,-1


In [21]:
curr_season_table['Actual Points'] = 0 
curr_season_table['Predicted Points'] = 0 

def make_actual_table(x, ht, at, which):
    if x == -1:
        curr_season_table.loc[ht, which] += 3
    elif x == 0:
        curr_season_table.loc[ht, which] += 1
        curr_season_table.loc[at, which] += 1
    elif x == 1:
        curr_season_table.loc[at, which] += 3

fixture_list.apply(lambda x: make_actual_table(x.Result, x.HomeTeam, x.AwayTeam, 'Actual Points'), axis=1)
fixture_list.apply(lambda x: make_actual_table(x.Predicted, x.HomeTeam, x.AwayTeam, 'Predicted Points'), axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
375    None
376    None
377    None
378    None
379    None
Length: 380, dtype: object

In [22]:
actual_index = curr_season_table.sort_values(by='Actual Points', ascending =0).index
predicted_index = curr_season_table.sort_values(by='Predicted Points', ascending =0).index
curr_season_table.sort_values(by='Actual Points', ascending =0)

,Actual Points,Predicted Points
Team,,
Man City,89,111
Arsenal,84,105
Man United,75,82
Newcastle,71,103
Liverpool,67,93
Brighton,62,84
Aston Villa,61,66
Tottenham,60,75
Brentford,59,78


In [23]:
acc(actual_index,predicted_index)
curr_season_table.sort_values(by='Predicted Points', ascending =0)

Accuracy of Predicted Table to Accurate Table 20.0%


,Actual Points,Predicted Points
Team,,
Man City,89,111
Arsenal,84,105
Newcastle,71,103
Liverpool,67,93
Brighton,62,84
Man United,75,82
Brentford,59,78
Tottenham,60,75
Aston Villa,61,66


In [24]:
curr_team_data = curr_team_data.rename(index={
    'Sheffield United':'Sheffield Utd',
    'Man United': 'Man Utd',
    'Tottenham':'Spurs'
})
print(curr_team_data.index.unique())
curr_team_data

Index(['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford', 'Brighton',
       'Chelsea', 'Crystal Palace', 'Everton', 'Fulham', 'Leeds', 'Leicester',
       'Liverpool', 'Man City', 'Man Utd', 'Newcastle', 'Nott'm Forest',
       'Southampton', 'Spurs', 'West Ham', 'Wolves'],
      dtype='object', name='Team')


,HAS,HAC,AAS,AAC
Team,,,,
Arsenal,0.840909,0.555556,1.000000,0.071429
Aston Villa,0.386364,0.407407,0.291667,0.321429
Bournemouth,0.090909,0.666667,0.250000,0.964286
Brentford,0.431818,0.296296,0.500000,0.428571
Brighton,0.477273,0.407407,1.000000,0.571429
Chelsea,0.090909,0.333333,0.291667,0.428571
Crystal Palace,0.113636,0.481481,0.333333,0.357143
Everton,0.000000,0.629630,0.291667,0.500000
Fulham,0.340909,0.703704,0.541667,0.285714


In [25]:
curr_fixtures = pd.read_csv(loc + "epl-2023.csv", usecols=['Home Team', 'Away Team'])
curr_fixtures.columns = ['HomeTeam', 'AwayTeam']

curr_fixtures = curr_fixtures.merge(curr_team_data['HAS'], left_on='HomeTeam', right_index=True, how='left')
curr_fixtures = curr_fixtures.merge(curr_team_data['HAC'], left_on='HomeTeam', right_index=True, how='left') 
curr_fixtures = curr_fixtures.merge(curr_team_data['AAS'], left_on='AwayTeam', right_index=True, how='left') 
curr_fixtures = curr_fixtures.merge(curr_team_data['AAC'], left_on='AwayTeam', right_index=True, how='left') 

curr_fixtures['HAtS'] = curr_fixtures['HAS'] * curr_fixtures['AAC']; # Home Attack Strength
curr_fixtures['AAtS'] = curr_fixtures['HAC'] * curr_fixtures['AAS']; # Away Attack Strength

print(curr_fixtures.shape)

fixture_list = curr_fixtures
curr_fixtures

(380, 8)


,HomeTeam,AwayTeam,HAS,HAC,AAS,AAC,HAtS,AAtS
0,Burnley,Man City,NaN,NaN,0.958333,0.000000,NaN,NaN
1,Arsenal,Nottingham Forest,0.840909,0.555556,NaN,NaN,NaN,NaN
2,Bournemouth,West Ham,0.090909,0.666667,0.208333,0.535714,0.048701,0.138889
3,Brighton,Luton,0.477273,0.407407,NaN,NaN,NaN,NaN
4,Everton,Fulham,0.000000,0.629630,0.541667,0.285714,0.000000,0.341049
...,...,...,...,...,...,...,...,...
375,Crystal Palace,Aston Villa,0.113636,0.481481,0.291667,0.321429,0.036526,0.140432
376,Liverpool,Wolves,0.681818,0.259259,0.041667,0.785714,0.535714,0.010802
377,Luton,Fulham,NaN,NaN,0.541667,0.285714,NaN,NaN
378,Man City,West Ham,1.000000,0.259259,0.208333,0.535714,0.535714,0.054012


In [26]:
# col_list = [x for x in col_list if x not in ['Result']]
# curr_fixtures = curr_fixtures[col_list]
# curr_fixtures

,HAS,HAC,AAS,AAC,HAtS,AAtS
0,NaN,NaN,0.958333,0.000000,NaN,NaN
1,0.840909,0.555556,NaN,NaN,NaN,NaN
2,0.090909,0.666667,0.208333,0.535714,0.048701,0.138889
3,0.477273,0.407407,NaN,NaN,NaN,NaN
4,0.000000,0.629630,0.541667,0.285714,0.000000,0.341049
...,...,...,...,...,...,...
375,0.113636,0.481481,0.291667,0.321429,0.036526,0.140432
376,0.681818,0.259259,0.041667,0.785714,0.535714,0.010802
377,NaN,NaN,0.541667,0.285714,NaN,NaN
378,1.000000,0.259259,0.208333,0.535714,0.535714,0.054012


In [27]:
# predicted = model1.predict(curr_fixtures)
# predicted
# predictedDF = pd.DataFrame(predicted, columns=['Predicted'])
# predictedDF.head()

ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values